In [12]:
import pandas as pd

# Load the dataset
file_path = r"C:\Users\USER\Desktop\음식점 리뷰 데이터\오리지널_리뷰.xlsx"
df = pd.read_excel(file_path)

# Step 1: Calculate frequencies of '대기시간' and '동반자' by '가게명'
df_grouped = df.groupby(['가게명', '대기시간', '동반자']).size().reset_index(name='빈도수')

# Step 2: Analyze '동반자' and '대기시간' for each '가게명'
df_companion = df_grouped.groupby(['가게명', '동반자'])['빈도수'].sum().reset_index()
df_wait_time = df_grouped.groupby(['가게명', '대기시간'])['빈도수'].sum().reset_index()

# Rank within each 가게명
df_companion['순위'] = df_companion.groupby('가게명')['빈도수'].rank(method='dense', ascending=False)
df_wait_time['순위'] = df_wait_time.groupby('가게명')['빈도수'].rank(method='dense', ascending=False)

# Filter for the top 2 ranks
top_companions = df_companion[df_companion['순위'] <= 2].copy()
top_wait_times = df_wait_time[df_wait_time['순위'] <= 2].copy()

# Compute totals for proportions
total_companions = df_companion.groupby('가게명')['빈도수'].sum().reset_index(name='총빈도수')
total_wait_times = df_wait_time.groupby('가게명')['빈도수'].sum().reset_index(name='총빈도수')

# Merge totals for proportions
top_companions = top_companions.merge(total_companions, on='가게명')
top_wait_times = top_wait_times.merge(total_wait_times, on='가게명')

# Calculate proportions
top_companions['비율'] = top_companions['빈도수'] / top_companions['총빈도수']
top_wait_times['비율'] = top_wait_times['빈도수'] / top_wait_times['총빈도수']

# Resolve duplicates by keeping the first entry of each rank per store
top_companions = top_companions.sort_values(by=['가게명', '순위', '빈도수'], ascending=[True, True, False])
top_companions = top_companions.drop_duplicates(subset=['가게명', '순위'], keep='first')

top_wait_times = top_wait_times.sort_values(by=['가게명', '순위', '빈도수'], ascending=[True, True, False])
top_wait_times = top_wait_times.drop_duplicates(subset=['가게명', '순위'], keep='first')

# Pivot for the top 2 rankings
pivot_companions = top_companions.pivot(index='가게명', columns='순위', values=['동반자', '비율']).reset_index()
pivot_wait_times = top_wait_times.pivot(index='가게명', columns='순위', values=['대기시간', '비율']).reset_index()

# Flatten multi-index columns
pivot_companions.columns = ['가게명', '동반자 1순위', '동반자 2순위', '동반자 1순위 비율', '동반자 2순위 비율']
pivot_wait_times.columns = ['가게명', '대기시간 1순위', '대기시간 2순위', '대기시간 1순위 비율', '대기시간 2순위 비율']

# Extract unique metadata for each 가게명
store_metadata = df[['가게명', '주소', '가게분류']].drop_duplicates()

# Merge all data
final_result = store_metadata.merge(pivot_companions, on='가게명', how='left')
final_result = final_result.merge(pivot_wait_times, on='가게명', how='left')

# Display or save the result
print(final_result)
# Save to Excel
final_result.to_excel('Top_Companions_and_Wait_Times_by_Store.xlsx', index=False)


              가게명                                    주소 가게분류 동반자 1순위 동반자 2순위  \
0      Just Steak                   서울 강남구 언주로152길 11-7   양식  연인・배우자      친구   
1      Lu307 이자카야                서울 강남구 압구정로10길 30-7 1층   일식      친구  연인・배우자   
2    갓잇 압구정 도산공원점  서울 강남구 언주로164길 24 아크로스빌딩 지상1층 GODEAT   양식     NaN     NaN   
3    갓포 모로이 도산공원점           서울 강남구 언주로172길 30 1층 갓포 모로미   일식  연인・배우자      친구   
4          강남철판요리          서울 강남구 압구정로28길 9-2 1층 강남철판요리   일식  연인・배우자      친구   
..            ...                                   ...  ...     ...     ...   
778          타이거풀                 서울 성동구 성수이로7가길 7 타이거풀   양식  연인・배우자      친구   
779          티엔티엔                   서울 성동구 왕십리로10길 5 1층   중식  연인・배우자      친구   
780      해피베네핏 성수              서울 성동구 성수이로7길 27 지하 102호   중식      친구  연인・배우자   
781           홍성집                 서울 성동구 왕십리로5길 12 1~3층   한식  연인・배우자   지인・동료   
782            희릿                   서울 성동구 왕십리로2길 34 1층   양식      친구  연인・배우자   

    동반자 1순위 비율 동반자 2순위 비율     대기시간 1순위 

In [13]:
final_result

,가게명,주소,가게분류,동반자 1순위,동반자 2순위,동반자 1순위 비율,동반자 2순위 비율,대기시간 1순위,대기시간 2순위,대기시간 1순위 비율,대기시간 2순위 비율
0,Just Steak,서울 강남구 언주로152길 11-7,양식,연인・배우자,친구,0.625,0.125,대기 시간 바로 입장,대기 시간 10분 이내,0.958333,0.041667
1,Lu307 이자카야,서울 강남구 압구정로10길 30-7 1층,일식,친구,연인・배우자,0.531915,0.244681,대기 시간 바로 입장,대기 시간 30분 이내,0.925532,0.042553
2,갓잇 압구정 도산공원점,서울 강남구 언주로164길 24 아크로스빌딩 지상1층 GODEAT,양식,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,갓포 모로이 도산공원점,서울 강남구 언주로172길 30 1층 갓포 모로미,일식,연인・배우자,친구,0.3375,0.3125,대기 시간 바로 입장,대기 시간 10분 이내,0.9375,0.05
4,강남철판요리,서울 강남구 압구정로28길 9-2 1층 강남철판요리,일식,연인・배우자,친구,0.545455,0.181818,대기 시간 바로 입장,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
778,타이거풀,서울 성동구 성수이로7가길 7 타이거풀,양식,연인・배우자,친구,0.340909,0.318182,대기 시간 바로 입장,대기 시간 10분 이내,0.988636,0.011364
779,티엔티엔,서울 성동구 왕십리로10길 5 1층,중식,연인・배우자,친구,0.375,0.258621,대기 시간 바로 입장,대기 시간 10분 이내,0.961207,0.030172
780,해피베네핏 성수,서울 성동구 성수이로7길 27 지하 102호,중식,친구,연인・배우자,0.447964,0.330317,대기 시간 바로 입장,대기 시간 10분 이내,0.963801,0.036199
781,홍성집,서울 성동구 왕십리로5길 12 1~3층,한식,연인・배우자,지인・동료,0.345794,0.196262,대기 시간 바로 입장,대기 시간 10분 이내,0.953271,0.046729


In [15]:
import pandas as pd

# Load the original and review datasets
review_file_path = r"C:\Users\USER\Desktop\음식점 리뷰 데이터\전체 4개국 통합.xlsx"

# Assuming result_df contains the grouped data analysis (e.g., 동반자, 대기시간 분석)
result_df = final_result

# Load the review dataset
review_df = pd.read_excel(review_file_path)

# Merge the result with reviews based on '가게명'
merged_df = result_df.merge(review_df[['가게명', '리뷰']], on='가게명', how='left')


In [16]:
# Save the merged DataFrame to a new file or display
output_file_path = '최최종빈도수파악.xlsx'
merged_df.to_excel(output_file_path, index=False)

# Print or display the resulting DataFrame
print("Merged results have been saved to:", output_file_path)

Merged results have been saved to: 최최종빈도수파악.xlsx


In [6]:
output_path = '빈도수파악.xlsx'
final_result.to_excel(output_path, index=False)

print(f"요약된 결과 파일이 저장되었습니다: {output_path}")

요약된 결과 파일이 저장되었습니다: 빈도수파악.xlsx


In [10]:
# Extract the longest review for each store
longest_reviews_all = df.groupby('가게명').apply(
    lambda x: x.loc[x['리뷰'].str.len().idxmax()] if x['리뷰'].notnull().any() else None
).reset_index(drop=True)

# Keep only the relevant columns
longest_reviews_all = longest_reviews_all[['가게명', '리뷰']].rename(columns={'리뷰': '리뷰 (Longest)'})

# Merge the longest review back into the aggregated data
final_data_with_longest_reviews = final_result.merge(longest_reviews_all, on='가게명', how='left')

# Save the updated dataset to Excel
final_data_with_longest_reviews.to_excel('Stores_with_Longest_Reviews.xlsx', index=False)

# Print the result for quick inspection
print(final_data_with_longest_reviews.head())

            가게명                                     주소 가게분류 동반자 1순위 동반자 2순위  \
0  37.5 시그니처 성수  서울 성동구 연무장길 41-19 1,2,3층 37.5 시그니처 성수   양식      친구  연인・배우자   
1      GTS 포레스트                     서울 성동구 서울숲2길 13 2층   양식  연인・배우자      친구   
2       갓잇 서울숲점         서울 성동구 왕십리로10길 9-14 1층 갓잇 서울숲점   양식      친구  연인・배우자   
3           계자람                 서울 성동구 성덕정17길 5 1층 계자람   한식  연인・배우자      친구   
4      규카츠정 성수점               서울 성동구 아차산로 96 지하1층 101호   일식  연인・배우자      친구   

  동반자 1순위 비율 동반자 2순위 비율     대기시간 1순위      대기시간 2순위 대기시간 1순위 비율 대기시간 2순위 비율  \
0   0.480226    0.20904  대기 시간 바로 입장  대기 시간 30분 이내    0.977401    0.016949   
1        0.3   0.266667  대기 시간 바로 입장  대기 시간 10분 이내    0.991667    0.008333   
2   0.379121   0.373626  대기 시간 바로 입장  대기 시간 10분 이내    0.956044    0.038462   
3      0.375       0.25  대기 시간 바로 입장  대기 시간 10분 이내    0.791667    0.166667   
4   0.384615   0.341346  대기 시간 바로 입장  대기 시간 30분 이내    0.288462    0.274038   

                                        리뷰 (Longest)  
0

C:\Users\USER\AppData\Local\Temp\ipykernel_33084\4266586037.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  longest_reviews_all = df.groupby('가게명').apply(


In [11]:
final_data_with_longest_reviews

,가게명,주소,가게분류,동반자 1순위,동반자 2순위,동반자 1순위 비율,동반자 2순위 비율,대기시간 1순위,대기시간 2순위,대기시간 1순위 비율,대기시간 2순위 비율,리뷰 (Longest)
0,37.5 시그니처 성수,"서울 성동구 연무장길 41-19 1,2,3층 37.5 시그니처 성수",양식,친구,연인・배우자,0.480226,0.20904,대기 시간 바로 입장,대기 시간 30분 이내,0.977401,0.016949,성수역인근에 위치해 있어서 접근성 좋고 하얀 인테리어가 너무 예뻐서 크리스마스 큰 ...
1,GTS 포레스트,서울 성동구 서울숲2길 13 2층,양식,연인・배우자,친구,0.3,0.266667,대기 시간 바로 입장,대기 시간 10분 이내,0.991667,0.008333,성수 미국여행 하는 느낌의 분위기도 너무 좋고 맛도 최고인 서울숲맛집 GTS포레스트...
2,갓잇 서울숲점,서울 성동구 왕십리로10길 9-14 1층 갓잇 서울숲점,양식,친구,연인・배우자,0.379121,0.373626,대기 시간 바로 입장,대기 시간 10분 이내,0.956044,0.038462,세트메뉴로 b세트 먹었는데 양이 엄청 푸짐해요~! 둘이서 먹으면 배 터져요! 세트로...
3,계자람,서울 성동구 성덕정17길 5 1층 계자람,한식,연인・배우자,친구,0.375,0.25,대기 시간 바로 입장,대기 시간 10분 이내,0.791667,0.166667,생기고 언제가지하고 생각하다가 이사 얼마 안남은 시점에서 갔는데 너무 맛있다 이 집...
4,규카츠정 성수점,서울 성동구 아차산로 96 지하1층 101호,일식,연인・배우자,친구,0.384615,0.341346,대기 시간 바로 입장,대기 시간 30분 이내,0.288462,0.274038,규카츠 엄청 좋아하는데 맛있는 곳이 없어서 아쉬워하다가 맛집 발견 .. ✨ 🥩 규카...
5,금금 성수점,서울 성동구 성수이로12길 11 1층 101호,한식,친구,연인・배우자,0.421053,0.240602,대기 시간 바로 입장,대기 시간 30분 이내,0.721805,0.12782,여자친구랑 성수 데이트 계획하며 찾아봤던 음식점 후보 중에 1곳이에요! 주말 오후 ...
6,기후,서울 성동구 성수이로 66 서울숲드림타워 1층 106호 기후,일식,연인・배우자,친구,0.515152,0.262626,대기 시간 바로 입장,대기 시간 10분 이내,0.969697,0.030303,기념일인 크리스마스에 미리 예약하고 고심하여 선택하여 갔습니다.장점-일단 음식 가짓...
7,꾸야 성수점,서울 성동구 왕십리로 94-2 2층,아시안,연인・배우자,친구,0.41704,0.295964,대기 시간 바로 입장,대기 시간 10분 이내,0.408072,0.318386,이전하고는 처음 방문했어요 연휴첫날 첫손님으로 방문했는데 여자 직원분인지 알바분인지...
8,난포,서울 성동구 서울숲4길 18-8 지층,한식,친구,연인・배우자,0.461538,0.260355,대기 시간 바로 입장,대기 시간 30분 이내,0.461538,0.16568,입장- 하루 전 날 오후 5시 예약으로 브레이크 타임 끝나자마자 첫 번째로 입장해서...
9,뇨끼룸,서울 성동구 성수일로1길 8 1층,양식,연인・배우자,친구,0.425234,0.401869,대기 시간 바로 입장,NaN,1.0,NaN,예약없이 가서 입구 옆 구석 좁은 곳에 앉아서 먹었는데요. 일단 의자가 너무 불편해...


In [12]:
output_path = '빈도수파악_리뷰추가.xlsx'
final_data_with_longest_reviews.to_excel(output_path, index=False)

print(f"요약된 결과 파일이 저장되었습니다: {output_path}")

요약된 결과 파일이 저장되었습니다: 빈도수파악_리뷰추가.xlsx


In [17]:
import pandas as pd

# File paths
freq_data_path = r"C:\Users\USER\Desktop\음식점 리뷰 데이터\데이터 셋만들기_ipynb\최최종빈도수파악.xlsx"
final_data_path = r"C:\Users\USER\Documents\카카오톡 받은 파일\최종데이터.xlsx"
output_file_path = '최종결과데이터셋파일.xlsx'

# Load datasets
freq_df = pd.read_excel(freq_data_path)
final_df = pd.read_excel(final_data_path)

# Merge the datasets on '가게명' to include '요약주소'
merged_df = freq_df.merge(final_df[['가게명', '요약주소']], on='가게명', how='left')

# Save the result to a new Excel file
merged_df.to_excel(output_file_path, index=False)

print("Merged results have been saved to:", output_file_path)


Merged results have been saved to: 최종결과데이터셋파일.xlsx
